In [ ]:
# (c) Westerveld 2023
# Watermark remover training and development using pytorch library with python
# A convolutional variational autoencoder that takes in an image with a watermark and (ideally) outputs the same image but without a watermark
# The architecture compresses the image then upsamples it, removing the watermark in the process and outputting an image.
# uses CLWD dataset

In [ ]:
!gdown 17y1gkUhIV6rZJg1gMG-gzVMnH27fm4Ij

!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive("CLWD.rar").extractall("")

Downloading...
From: https://drive.google.com/uc?id=17y1gkUhIV6rZJg1gMG-gzVMnH27fm4Ij
To: /content/CLWD.rar
100% 3.35G/3.35G [00:48<00:00, 69.0MB/s]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
# include libraries
#!pip install torchvision
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.io import read_image
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torchvision.transforms as T
import random
import torch.nn.functional as F

# check if gpu available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [ ]:
# custom dataset - length 60 000, returns watermarked and watermark free image
class Coles(Dataset):
  def __init__(self, watermark_free, watermarked, transform = None, target_transform = None):
    #self.watermark_free = Image.open("CLWD/train/Watermark_image" + watermark_free)
    #self.img_dir = watermarked
    self.transform = transform
    self.target_transform = target_transform
  # C:\Users\Cole\Desktop/CLWD/
  def __getitem__(self, idx):
    img_path = r"C:\Users\Cole\Desktop/CLWD/CLWD/train/Watermarked_image/" + str(idx + 1) + ".jpg"
    watermarked = Image.open(img_path).convert('RGB')
    img_path = r"C:\Users\Cole\Desktop/CLWD/CLWD/train/Watermark_free_image/" + str(idx + 1) + ".jpg"
    watermark_free = Image.open(img_path).convert('RGB')
    transform = T.Resize(size = (256,256))

    transform_one = T.ToTensor()
    img = torch.FloatTensor()
    label = torch.FloatTensor()


    #img.to(device)
    #label.to(device)
    img = transform_one(watermarked)
    label = transform_one(watermark_free)

    img = transform(img)
    label = transform(label)


    # watermarked.cuda()
    # watermark_free.cuda()


    return img, label

  def __len__(self):
    #hardcoded for now
    return 60_000

Train_dataset = Coles("", "")

In [ ]:
# Test traing the discriminator - give watermarked and watermark free images from the dataset and have it classify them as either waterrmarked or not


class Discrim_dataset(Dataset):
  def __init__(self, watermark_free, watermarked, transform = None, target_transform = None):
    #self.watermark_free = Image.open("CLWD/train/Watermark_image" + watermark_free)
    #self.img_dir = watermarked
    self.transform = transform
    self.target_transform = target_transform
  # C:\Users\Cole\Desktop/CLWD/
  def __getitem__(self, idx):
    # pick randomly watermarked or not
    num = random.randint(0, 1)
    if num == 0:
      img_path = r"CLWD/train/Watermarked_image/" + str(idx + 1) + ".jpg"
      label = 1
    else:
      img_path = r"CLWD/train/Watermark_free_image/" + str(idx + 1) + ".jpg"
      label = 0

    image = Image.open(img_path).convert('RGB')
    transform = T.Resize(size = (256,256))

    transform_one = T.ToTensor()
    img = torch.FloatTensor()



    #img.to(device)
    #label.to(device)
    img = transform_one(image)

    img = transform(img)

    # watermarked.cuda()
    # watermark_free.cuda()

    return img, label

  def __len__(self):
    #hardcoded for now
    return 60_000

discriminator_dataset = Discrim_dataset("", "")

In [ ]:
# Defining the model - encoder and decoder convolutional part

class Encoder(nn.Module):
  def __init__(self):
    super().__init__()
    #takes in 64*64 (or as much as I reasonably can) by 3 channels
    self.encoder = nn.Sequential(
        nn.Conv2d(3, 64, 3, stride = 2, padding = 1),
        nn.ReLU(),
        nn.Conv2d(64, 128, 3, stride = 2, padding = 1),
        nn.ReLU(),
        nn.Conv2d(128, 256, 3, stride = 2, padding = 1),
        nn.ReLU(),
        nn.Conv2d(256, 512, 3, stride = 2, padding = 1),


    )
        # this may not be perfect but this is how i think it is right now
        # conv2d(in_channels, out_channels, kernel size, stride, padding)
        # in channels is the number of channels that are input, for this case 3 because R, G, B
        # out channels is number of output(width of the matrix?), will be batch size - number of images
        # kernel size, area of sqare (or rectangle if tuple is provided) that is used to convolve pixel
        # kernel size will probably just be 3, meaning thagt if x is the convoluted pixel then it would take from pixels like this:
        # PPP
        # PxP
        # PPP
        # stride is how many pixels it skips, should probably just be 1
        # padding is how much it leaves off the edges, should be 1

    self.decoder = nn.Sequential(

        nn.ConvTranspose2d(512, 256, 3, stride = 2, padding = 1, output_padding = 1),
        nn.ReLU(),
        nn.ConvTranspose2d(256, 128, 3, stride = 2, padding = 1, output_padding = 1),
        nn.ReLU(),
        nn.ConvTranspose2d(128, 64, 3, stride = 2, padding = 1, output_padding = 1),
        nn.ReLU(),
        nn.ConvTranspose2d(64, 3, 3, stride = 2, padding = 1, output_padding = 1),
        nn.Sigmoid()

    )

  # output
  def forward(self, x):
    #print("began")
    encoded = self.encoder(x)
    #print(encoded)
    decoded = self.decoder(encoded)
    return decoded


In [ ]:
# # CNN model from here, adjsted a bit
# # https://medium.com/thecyphy/train-cnn-model-with-pytorch-21dafb918f48

# class Discriminator(nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.Network = nn.Sequential(
#         nn.Conv2d(3, 8, 2, stride = 2, padding = 1),
#         nn.LeakyReLU(0.2),
#         nn.Conv2d(8, 16, 2, stride = 2,  padding = 1),


#         nn.Conv2d(16, 32, 2, stride = 2,  padding = 1),
#         nn.LeakyReLU(0.2),
#         nn.Conv2d(32, 64, 2, stride = 2,  padding = 1),

#         nn.Conv2d(64, 128, 2, stride = 2,  padding = 1),
#         nn.LeakyReLU(0.2),
#         nn.Conv2d(128, 1, 2, stride = 2,  padding = 1),
#         nn.LeakyReLU(0.2),
#         nn.Conv2d(128, 1, 2, stride = 2,  padding = 1),
#         nn.Sigmoid(),


    )
    # self.conv1 = nn.Conv2d(3, 6, 5)
    # self.pool = nn.MaxPool2d(2, 2)
    # self.conv2 = nn.Conv2d(6, 16, 5)




  def forward(self, x):
      return self.Network(x)


In [ ]:
# train_discrim_dataloader = DataLoader(discriminator_dataset, batch_size=16, shuffle=True)
# dataiter = iter(train_discrim_dataloader)
# images, labels = next(dataiter)
# print(images.shape)
# var_1 = nn.Conv2d(3, 6, 5)
# var_2 = nn.ReLU()
# var_3 = nn.Conv2d(6, 16, 3)
# var_4 = nn.ReLU()
# pool = nn.MaxPool2d(2, 2)
# var_5 = nn.Conv2d(16, 32, 5)
# var_6 = nn.ReLU()
# var_7 = nn.Conv2d(32, 64, 3)


# var_9 = nn.Conv2d(64, 128, 5)
# var_10 = nn.ReLU()
# var_11 = nn.Conv2d(128, 256, 3)
# var_13 = nn.Sigmoid()


# x = var_1(images)
# print(x.shape)
# x = var_2(x)
# print(x.shape)
# x = var_3(x)
# print(x.shape)

# x = pool(x)
# print(x.shape)

# x = var_5(x)
# print(x.shape)
# x = var_6(x)
# print(x.shape)
# x = var_7(x)
# print(x.shape)

# x = pool(x)
# print(x.shape)


# x = var_9(x)
# print(x.shape)
# x = var_10(x)
# print(x.shape)
# x = var_11(x)
# print(x.shape)
# x = pool(x)
# print(x.shape)





In [ ]:
# testest

train_discrim_dataloader = DataLoader(discriminator_dataset, batch_size=16, shuffle=True)
dataiter = iter(train_discrim_dataloader)
images, labels = next(dataiter)
print(images.shape)


one = nn.Conv2d(3, 6, 2, stride = 2, padding = 1)
two = nn.LeakyReLU(0.2)
three = nn.Conv2d(6, 16, 2, stride = 2,  padding = 1)


four = nn.Conv2d(16, 32, 2, stride = 2,  padding = 1)
five = nn.LeakyReLU(0.2)
six = nn.Conv2d(32, 64, 2, stride = 2,  padding = 1)

seven = nn.Conv2d(64, 128, 2, stride = 2,  padding = 1)
eight = nn.LeakyReLU(0.2)
nine = nn.Conv2d(128, 1, 2, stride = 2,  padding = 1)

eleven = nn.Flatten(0, -1)
twelve = nn.Linear(400, 16)
ten = nn.Sigmoid()

x = one(images)
print(x.shape)
x = two(x)
print(x.shape)
x = three(x)
print(x.shape)

x = four(x)
print(x.shape)
x = five(x)
print(x.shape)
x = six(x)
print(x.shape)

x = seven(x)
print(x.shape)
x = eight(x)
print(x.shape)
x = nine(x)
print(x.shape)


x = eleven(x)
print(x.shape)

x = twelve(x)
print(x.shape)

x = ten(x)
print(x.shape)
print(x)


torch.Size([16, 3, 256, 256])
torch.Size([16, 6, 129, 129])
torch.Size([16, 6, 129, 129])
torch.Size([16, 16, 65, 65])
torch.Size([16, 32, 33, 33])
torch.Size([16, 32, 33, 33])
torch.Size([16, 64, 17, 17])
torch.Size([16, 128, 9, 9])
torch.Size([16, 128, 9, 9])
torch.Size([16, 1, 5, 5])
torch.Size([400])
torch.Size([16])
torch.Size([16])
tensor([0.4990, 0.4854, 0.4916, 0.4978, 0.4995, 0.5085, 0.5009, 0.5157, 0.4997,
        0.5086, 0.4973, 0.5045, 0.4908, 0.5172, 0.4993, 0.4853],
       grad_fn=<SigmoidBackward0>)


In [ ]:
# # initializing the model
# model = Encoder()
# discrim_model = Discriminator()
# criterion = nn.MSELoss()
# optimizer = torch.optim.SGD(discrim_model.parameters(), lr = 1e-3)


NameError: ignored

In [ ]:
# CNN model from here, adjsted a bit
# https://medium.com/thecyphy/train-cnn-model-with-pytorch-21dafb918f48

class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.Network = nn.Sequential(
        nn.Conv2d(3, 8, 2, stride = 2, padding = 1),
        nn.LeakyReLU(0.2),
        nn.Conv2d(8, 16, 2, stride = 2,  padding = 1),
        nn.BatchNorm2d(16),

        nn.Conv2d(16, 32, 2, stride = 2,  padding = 1),
        nn.LeakyReLU(0.2),
        nn.Conv2d(32, 64, 2, stride = 2,  padding = 1),
        nn.BatchNorm2d(64),

        nn.Conv2d(64, 128, 2, stride = 2,  padding = 1),
        nn.LeakyReLU(0.2),
        nn.Conv2d(128, 1, 2, stride = 2,  padding = 1),
        nn.BatchNorm2d(1),

        nn.Flatten(0, -1),
        nn.Linear(400, 16),
        nn.Sigmoid()


    )
    # self.conv1 = nn.Conv2d(3, 6, 5)
    # self.pool = nn.MaxPool2d(2, 2)
    # self.conv2 = nn.Conv2d(6, 16, 5)




  def forward(self, x):

      return self.Network(x)

# initializing the model
model = Encoder()
discrim_model = Discriminator()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(discrim_model.parameters(), lr = 1e-2)

# practice training loop for the discriminator
train_discrim_dataloader = DataLoader(discriminator_dataset, batch_size=16, shuffle=True, num_workers = 8)

discrim_model.to(device)
model.to(device)
criterion.to(device)
#optimizer.to(device)
# stuffs = T.Resize(size = (256,256))
# optimus_prime = T.ToPILImage()
# pic123 = Image.open("Downloads/images.jpg").convert('RGB')
# transform_three = T.ToTensor()
# img = transform_three(pic123)
# img = torch.FloatTensor(img)
# img = stuffs(img)
# img = img.to(device)

# print(img.shape)

#transform = T.Resize(size = (256,256))
transform = T.ToPILImage()

# Training loop
epochs = 12
for e in range(epochs):
    counter = 0
    running_loss = 0
    for images, labels in train_discrim_dataloader:
        counter += 1
        if counter % 37.5 == 0:
          print("training at epoch ", e, " ", (counter / 3750) * 100, "% done")
        # print(images.shape)
        # images = images.reshape(-1, 256*256) * 0.00390625
        # print(images.shape)
        # print(images)
        # images = images.float()
        # print(images.shape)
        # print(images)
        labels = labels.to(device)

        #print(len(images))
        #batch_size, channels, depth, height, width

        #images = np.reshape(images, (len(images), 3, 256, 256)) /256

        images = images.to(device)
        #images = torch.cat([images[0:-1], images[-1+1:]])


        #images = np.reshape(images, (len(images), 3, 256, 256)) /256


        #images = images.float()
        #print(images)
        #print(images.shape)

        # Training pass

        #print(type(images))
        # test123 = labels[0]
        # pic2 = transform(test123)
        # pic2.show()

        # test1234 = images[0]
        # pic3 = transform(test1234)
        # pic3.show()
        #print(images.shape)
        output = discrim_model(images)
        #print(output.shape)


        output.to(device)
        #print(output.shape)
        #print(labels.shape)
        #print(output)
        #print(labels)
        output = output.float()
        labels = labels.float()


        optimizer.zero_grad()
        #log_softmax = nn.LogSoftmax(dim = -1)
        #print(log_softmax(output).shape)
        #print("output cuda: ", output.is_cuda)
        #print("labels cuda: ", labels.is_cuda)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()



    # output = discrim_model(img)
    # output = output * 255
    # pic1 = transform(output)
    # pic1.show()
    torch.save({
            'epoch': e,
            'model_state_dict': discrim_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f"discrim_4_epoch-{e}.pth")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - P

training at epoch  0   2.0 % done
training at epoch  0   4.0 % done
training at epoch  0   6.0 % done
training at epoch  0   8.0 % done
training at epoch  0   10.0 % done
training at epoch  0   12.0 % done
training at epoch  0   14.000000000000002 % done
training at epoch  0   16.0 % done
training at epoch  0   18.0 % done
training at epoch  0   20.0 % done
training at epoch  0   22.0 % done
training at epoch  0   24.0 % done
training at epoch  0   26.0 % done
training at epoch  0   28.000000000000004 % done
training at epoch  0   30.0 % done
training at epoch  0   32.0 % done
training at epoch  0   34.0 % done
training at epoch  0   36.0 % done
training at epoch  0   38.0 % done
training at epoch  0   40.0 % done
training at epoch  0   42.0 % done
training at epoch  0   44.0 % done
training at epoch  0   46.0 % done
training at epoch  0   48.0 % done
training at epoch  0   50.0 % done
training at epoch  0   52.0 % done
training at epoch  0   54.0 % done
training at epoch  0   56.00000

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

training at epoch  1   2.0 % done
training at epoch  1   4.0 % done
training at epoch  1   6.0 % done
training at epoch  1   8.0 % done
training at epoch  1   10.0 % done
training at epoch  1   12.0 % done
training at epoch  1   14.000000000000002 % done
training at epoch  1   16.0 % done
training at epoch  1   18.0 % done
training at epoch  1   20.0 % done
training at epoch  1   22.0 % done
training at epoch  1   24.0 % done
training at epoch  1   26.0 % done
training at epoch  1   28.000000000000004 % done
training at epoch  1   30.0 % done
training at epoch  1   32.0 % done
training at epoch  1   34.0 % done
training at epoch  1   36.0 % done
training at epoch  1   38.0 % done
training at epoch  1   40.0 % done
training at epoch  1   42.0 % done
training at epoch  1   44.0 % done
training at epoch  1   46.0 % done
training at epoch  1   48.0 % done
training at epoch  1   50.0 % done
training at epoch  1   52.0 % done
training at epoch  1   54.0 % done
training at epoch  1   56.00000

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

...

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

training at epoch  3   2.0 % done
training at epoch  3   4.0 % done
training at epoch  3   6.0 % done
training at epoch  3   8.0 % done
training at epoch  3   10.0 % done
training at epoch  3   12.0 % done
training at epoch  3   14.000000000000002 % done
training at epoch  3   16.0 % done
training at epoch  3   18.0 % done
training at epoch  3   20.0 % done
training at epoch  3   22.0 % done
training at epoch  3   24.0 % done
training at epoch  3   26.0 % done
training at epoch  3   28.000000000000004 % done
training at epoch  3   30.0 % done
training at epoch  3   32.0 % done
training at epoch  3   34.0 % done
training at epoch  3   36.0 % done
training at epoch  3   38.0 % done
training at epoch  3   40.0 % done
training at epoch  3   42.0 % done
training at epoch  3   44.0 % done
training at epoch  3   46.0 % done
training at epoch  3   48.0 % done
training at epoch  3   50.0 % done
training at epoch  3   52.0 % done
training at epoch  3   54.0 % done
training at epoch  3   56.00000

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

training at epoch  4   2.0 % done
training at epoch  4   4.0 % done
training at epoch  4   6.0 % done
training at epoch  4   8.0 % done
training at epoch  4   10.0 % done


KeyboardInterrupt: ignored

In [ ]:
# # initializing model object and dataset for training

# train_dataloader = DataLoader(discriminator_dataset, batch_size=16, shuffle=True)

# # move to gpu - im not gonna put this in yet until all other bugs are worked out
# # so as to not intrudce more until current ones are fixed
# #discrim_model.to(device)
# criterion.to(device)
# #optimizer.to(device)


# #transform = T.Resize(size = (256,256))
# transform = T.ToPILImage()

# # Training loop
# epochs = 12
# for e in range(epochs):
#     counter = 0
#     running_loss = 0
#     for images, labels in train_dataloader:
#         counter += 1
#         if counter % 37.5 == 0:
#           print("training at epoch ", e, " ", (counter / 3750) * 100, "% done")
#         labels = labels.to(device)
#         # print(images.shape)
#         # images = images.reshape(-1, 256*256) * 0.00390625
#         # print(images.shape)
#         # print(images)
#         # images = images.float()
#         # print(images.shape)
#         # print(images)

#         #print(len(images))
#         #batch_size, channels, depth, height, width

#         #images = np.reshape(images, (len(images), 3, 256, 256)) /256

#         images = images.to(device)
#         #images = torch.cat([images[0:-1], images[-1+1:]])


#         #images = np.reshape(images, (len(images), 3, 256, 256)) /256

#         #print(images.shape)
#         #images = images.float()
#         #print(images)
#         #print(images.shape)

#         # Training pass

#         #print(type(images))
#         # test123 = labels[0]
#         # pic2 = transform(test123)
#         # pic2.show()

#         # test1234 = images[0]
#         # pic3 = transform(test1234)
#         # pic3.show()

#         output = discrim_model(images)

#         output.to(device)
#         output = output.float()
#         labels = labels.float()

#         optimizer.zero_grad()

#         #print("output cuda: ", output.is_cuda)
#         #print("labels cuda: ", labels.is_cuda)
#         loss = criterion(output, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     transform = T.ToPILImage()
#     pic1 = transform(img)
#     pic1.show()
#     torch.save({
#             'epoch': e,
#             'model_state_dict': discrim_model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': loss,
#             }, f"discrim_1_epoch-{e}.pth")

training at epoch  0   2.0 % done
training at epoch  0   4.0 % done
training at epoch  0   6.0 % done
training at epoch  0   8.0 % done


KeyboardInterrupt: ignored

In [ ]:
# Display image and label.

# testing dataset
train_dataloader = DataLoader(discriminator_dataset, batch_size=16, shuffle=True)

train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0]#.squeeze()
label = train_labels[0]

transform = T.ToPILImage()
pic1 = transform(img)
pic1.show()

# label = train_features[0]#.squeeze()
# label = train_labels[0]
# pic2 = transform(label)
# pic2.show()

print(f"Label: {label}")

Feature batch shape: torch.Size([16, 3, 150, 150])
Labels batch shape: torch.Size([16])
Label: 0
